<a href="https://colab.research.google.com/github/ErikHartman/bopep/blob/main/bopep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# bopep: identifying peptide binders in large scale peptidomic data

Bayesian optimization guided search for binders in large scale peptidomic datasets.

Relies on ESM2 for peptide embeddings, ColabFold utilizing AlphaFold 2 multimer for docking and PyRosetta for interface energy calculations. A deep ensemble is used as a surrogate model utilizing Torch.

In [10]:
#@title Installation

import os, time, io
import contextlib

print("Fetching bopep")
!git clone https://github.com/ErikHartman/bopep /content/bopep/

print("Installing ColabFold")
!pip install --quiet colabfold
!pip install --quiet biopython

print("Installing esm-fair and fetching ESM model")
!pip install --quiet fair-esm

# Code to fetch ESM model if it doesn't already exist
esm_model_path = "/root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt"
if not os.path.exists(esm_model_path):
    print("Downloading ESM model...")
    import esm
    model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
else:
    print("ESM model already exists.")

print("Installing PyRosetta")
!pip install pyrosettacolabsetup
import pyrosettacolabsetup
pyrosettacolabsetup.install_pyrosetta(serialization=True, cache_wheel_on_google_drive=False)

print("Installing other necessary packages")
!pip install -r https://raw.githubusercontent.com/ErikHartman/bopep/main/requirements.txt


Fetching bopep
fatal: destination path '/content/bopep' already exists and is not an empty directory.
Installing ColabFold
Installing esm-fair and fetching ESM model
ESM model already exists.
Installing PyRosetta
Installing other necessary packages


In [ ]:
#@title Embedding settings

In [ ]:
#@title Generate embeddings

In [ ]:
#@title Bayesian optimization settings

In [ ]:
#@title Initialize PyRosetta

In [ ]:
#@title Run bopep!